In [88]:
#Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, GRU
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [89]:
#importing the dataset
from google.colab import files
uploaded=files.upload()

Saving ETH-USD (2).csv to ETH-USD (2) (2).csv


In [90]:
df = pd.read_csv('ETH-USD (2).csv')

In [91]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,320.884003,893249984
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,299.252991,885985984
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,314.681000,842300992
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984
...,...,...,...,...,...,...,...
2030,2023-06-01,1873.914673,1887.705322,1846.227417,1862.201416,1862.201416,5640027197
2031,2023-06-02,1862.423706,1910.278442,1851.964722,1907.256592,1907.256592,6097746022
2032,2023-06-03,1907.420776,1908.822266,1885.485474,1892.412476,1892.412476,3472274607
2033,2023-06-04,1892.398193,1912.205688,1885.651855,1890.513550,1890.513550,3747042696


In [92]:
#Get number of rows and colums in the data set
df.shape

(2035, 7)

In [93]:
#Create a new dataframe with only the 'Close column'
data = df.filter(['Close'])

#Convert the dataframe to a numpy array
dataset = data.values

#Get the number of rows to train the model
from sklearn.model_selection import train_test_split

# Splitting the dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Printing the sizes of the split datasets
print("Training data size:", len(train_data))
print("Testing data size:", len(test_data))

Training data size: 1628
Testing data size: 407


In [94]:
training_data_len = math.ceil(len(dataset) * .8)

training_data_len

1628

In [95]:
#Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

array([[0.0500395 ],
       [0.0454642 ],
       [0.04872747],
       ...,
       [0.38244261],
       [0.38204096],
       [0.3653978 ]])

In [96]:
#Create the training dataset
#Create the scaled training dataset

train_data = scaled_data[0:training_data_len , :]

#split the into x_train and y_train dataset
x_train = []
y_train = []

for i in range(60, len(train_data)):
  x_train.append(train_data[i-60:i, 0])
  y_train.append(train_data[i, 0])
  if i<= 61:
    print(x_train)
    print(y_train)
    print()

[array([0.0500395 , 0.0454642 , 0.04872747, 0.04729487, 0.0491579 ,
       0.05358176, 0.05267774, 0.05216312, 0.05247405, 0.05569289,
       0.0571257 , 0.05973666, 0.05839797, 0.06268138, 0.06892405,
       0.08261865, 0.08079221, 0.0818612 , 0.08377014, 0.08219371,
       0.07259534, 0.07673914, 0.08084805, 0.08019425, 0.08070273,
       0.08162304, 0.08015872, 0.0728206 , 0.07405162, 0.07862523,
       0.08232062, 0.07559844, 0.09112687, 0.11995542, 0.1308138 ,
       0.12934354, 0.12693903, 0.12942667, 0.13445355, 0.15024744,
       0.15705359, 0.1554171 , 0.15583526, 0.12491102, 0.13432919,
       0.12899074, 0.14415345, 0.14584601, 0.1435206 , 0.13805948,
       0.14156408, 0.13387866, 0.14222845, 0.14559324, 0.16924134,
       0.18579795, 0.18964797, 0.19320101, 0.20249926, 0.22608115])]
[0.22509971577339596]

[array([0.0500395 , 0.0454642 , 0.04872747, 0.04729487, 0.0491579 ,
       0.05358176, 0.05267774, 0.05216312, 0.05247405, 0.05569289,
       0.0571257 , 0.05973666, 0.05

In [97]:
#Convert the x_train and y_train to numpy arrays so we can use them in training our models
x_train, y_train = np.array(x_train), np.array(y_train)

In [98]:
#Reshape the data to fit model into:
#Number of samples, Number of timestamps, Number of features(Close since we are looking for closing price of data)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(1568, 60, 1)

In [104]:
#Build the GRU Model
model = Sequential()
model.add(GRU(50, return_sequences=True,input_shape=(x_train.shape[1], 1)))
model.add(GRU(50, return_sequences= False))
model.add(Dense(25))
model.add(Dense(1))

In [105]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 60, 50)            7950      
                                                                 
 gru_5 (GRU)                 (None, 50)                15300     
                                                                 
 dense_6 (Dense)             (None, 25)                1275      
                                                                 
 dense_7 (Dense)             (None, 1)                 26        
                                                                 
Total params: 24,551
Trainable params: 24,551
Non-trainable params: 0
_________________________________________________________________


In [106]:
#Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [107]:
#Train the model
model.fit(x_train, y_train, batch_size=1, epochs=100)

Epoch 1/100
1568/1568 [==============================] - 63s 38ms/step - loss: 0.0016
Epoch 2/100
1568/1568 [==============================] - 57s 36ms/step - loss: 7.6748e-04
Epoch 3/100
1306/1568 [=======================>......] - ETA: 10s - loss: 8.4038e-04

KeyboardInterrupt: ignored

In [ ]:
#Create Testing dataset
#Create a new array containing scaled values from index 1568 to 2034
test_data = scaled_data[training_data_len - 60:, :]
#Create the datasets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
 x_test.append(test_data[i-60:i, 0])
 

In [ ]:
#Convert data to numpy array
x_test = np.array(x_test)

In [ ]:
#Reshape the Test dataset
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
#Get the models predicted price values
predictions = model.predict(x_test)
#Unscaling values and making x_test and y_test have same values
predictions = scaler.inverse_transform(predictions)

In [ ]:
#Evaluate model
#Get the RMSE (root mean square error)
rmse = np.sqrt( np.mean( predictions - y_test)**2 )
rmse

In [ ]:
#Get the Mean Absolute Error (MAE)
mae = np.mean(np.abs(predictions - y_test))
mae

In [ ]:
#Get the Mean Squared Error (MSE)
mse = np.mean((predictions - y_test) ** 2)
mse

In [ ]:
#Get the R-squared (R2) Score
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
r2

In [ ]:
#Get the Explained Variance Score
from sklearn.metrics import explained_variance_score
evs = explained_variance_score(y_test, predictions)
evs

In [ ]:
#Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

#Visualize the data
plt.figure(figsize=(10,5))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val','Predictions'], loc='lower right')

In [ ]:
#Show the valid(actual closing price) and predicted prices
valid

In [ ]:
#Try and predict price of June 6th 2023
#Import dataset
from google.colab import files
uploaded=files.upload()

In [ ]:
df = pd.read_csv('ETH-USD (2).csv')

In [ ]:
#Predict the price of June 6th 2023
#create new dataframe
new_df = df.filter(['Close'])
#Get the last 60 days closing price values and convert the dataframe to an array
last_60_days = new_df[-60:].values
#Scale the data to be values between 0 and 1
last_60_days_scaled = scaler.transform(last_60_days)
#Create an empty list
X_test = []
#Append past 60 days
X_test.append(last_60_days_scaled)
#Convert the X_test to a numpy array
X_test = np.array(X_test)
#Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Get the predicted scaled price
pred_price = model.predict(X_test)
#undo the scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
#Import price of June 6th 2023
from google.colab import files
uploaded=files.upload()

In [ ]:
df = pd.read_csv('June 6th 2023.csv')

In [ ]:
#Print Closing Price of June 6th 2023
print(df['Close'])